# Privacy-QoS Trade-Off Analysis

In [1]:
import os
import pandas as pd
import numpy as np

Define method to get sub-datasets of the main dataset

In [2]:
def getSubDatasets(df):
    firstDf = df[(df["dummyLocation"]==False) & (df["gpsPerturbated"]==True)].reset_index(drop=True)
    secondDf = df[(df["dummyLocation"]==True) & (df["gpsPerturbated"]==False)].reset_index(drop=True)
    thirdDf = df[(df["dummyLocation"]==True) & (df["gpsPerturbated"]==True)].reset_index(drop=True)

    return [firstDf, secondDf, thirdDf]

Import privacy and qos datasets, define alphas

In [3]:
privacyDf = pd.read_csv(os.path.join('csvs', f'average_distances.csv'), index_col=[0])
qosDf = pd.read_csv(os.path.join('csvs', f'qos.csv'), index_col=[0])

CSV_NAMES = [os.path.join('csvs', f'tradeOffComplete.csv'), os.path.join('csvs', f'tradeOff.csv')]

for csv in CSV_NAMES:
    if os.path.exists(csv):
        os.remove(csv)

alphas =  list(map(lambda x: round(x, 5), [*np.arange(0.0, 1.0, 1/500), 1.0]))

privacySubDf = getSubDatasets(privacyDf)
qosSubDf = getSubDatasets(qosDf)

Linear combination to achieve the tradeoff between privacy and qos

In [4]:
def linearCombination(alpha, privacy, qos):
    return alpha * privacy + (1-alpha) * qos

Define min-max scaling function

In [5]:
def minMaxScaling(serie):
    min = np.min(serie)
    max = np.max(serie)

    return (serie - min) / (max - min)


Min-max scaling of privacy and qos values of the main dataset

In [6]:
pValues = privacyDf["avg"]

privacyDf["avg"] = minMaxScaling(privacyDf["avg"])
qosDf["mse"] = minMaxScaling([pow(el, -1) for el in qosDf["mse"]])

privacySD = getSubDatasets(privacyDf)
qosSD = getSubDatasets(qosDf)

Define trade-off getter

In [7]:
def getTradeOff(privacy, qos, csvName):
    df=privacy.copy()
    df.drop('avg', inplace=True, axis=1)
    
    for alpha in alphas:
        df["alpha"]=alpha
        df["tradeOff"]=[linearCombination(alpha, avg, mse) for avg, mse in zip(privacy["avg"], qos["mse"])]
        tradeOff = df[df["tradeOff"]==np.max(df["tradeOff"])]

        if not os.path.exists(csvName):
            tradeOff.to_csv(csvName)
        else:
            tradeOff.to_csv(csvName, mode='a', header=False)

Compute Privacy-QoS trade-off

In [13]:
privacyQos = getTradeOff(privacyDf, qosDf, CSV_NAMES[0])

privacyQosSD = []
for prv, qos in zip(privacySD, qosSD):
    privacyQosSD.append(getTradeOff(prv, qos, CSV_NAMES[1]))

tradeOffComplete = pd.read_csv(CSV_NAMES[0], index_col=0)
tradeOff = pd.read_csv(CSV_NAMES[1], index_col=0)

# In this dataset after the trade-off computation there aren't tradeOff to both true dummy location and true gps perturbated 
tradeOffComplete.reset_index(drop=True)

# print(tradeOffComplete.reset_index(drop=True).to_latex())

\begin{tabular}{lllrrrrr}
\toprule
{} &  dummyLocation &  gpsPerturbated &  dumRadMin &  dumRadStep &  pertDec &  alpha &  tradeOff \\
\midrule
0   &          False &            True &         -1 &          -1 &        5 &  0.000 &     1.000 \\
1   &          False &            True &         -1 &          -1 &        5 &  0.002 &     0.998 \\
2   &          False &            True &         -1 &          -1 &        5 &  0.004 &     0.996 \\
3   &          False &            True &         -1 &          -1 &        5 &  0.006 &     0.994 \\
4   &          False &            True &         -1 &          -1 &        5 &  0.008 &     0.992 \\
5   &          False &            True &         -1 &          -1 &        5 &  0.010 &     0.990 \\
6   &          False &            True &         -1 &          -1 &        5 &  0.012 &     0.988 \\
7   &          False &            True &         -1 &          -1 &        5 &  0.014 &     0.986 \\
8   &          False &            True &        

Only Dummy Location trade-offs of the sub-dataset

In [14]:
tradeOff[(tradeOff['dummyLocation'] == True) & (tradeOff['gpsPerturbated'] == False)].reset_index(drop=True)
# print(tradeOff[(tradeOff['dummyLocation'] == True) & (tradeOff['gpsPerturbated'] == False)].reset_index(drop=True).to_latex())

\begin{tabular}{lllrrrrr}
\toprule
{} &  dummyLocation &  gpsPerturbated &  dumRadMin &  dumRadStep &  pertDec &  alpha &      tradeOff \\
\midrule
0   &           True &           False &        500 &         250 &       -1 &  0.000 &  1.071027e-26 \\
1   &           True &           False &       3000 &        2000 &       -1 &  0.002 &  7.365314e-04 \\
2   &           True &           False &       3000 &        2000 &       -1 &  0.004 &  1.473063e-03 \\
3   &           True &           False &       3000 &        2000 &       -1 &  0.006 &  2.209594e-03 \\
4   &           True &           False &       3000 &        2000 &       -1 &  0.008 &  2.946125e-03 \\
5   &           True &           False &       3000 &        2000 &       -1 &  0.010 &  3.682657e-03 \\
6   &           True &           False &       3000 &        2000 &       -1 &  0.012 &  4.419188e-03 \\
7   &           True &           False &       3000 &        2000 &       -1 &  0.014 &  5.155719e-03 \\
8   &       

Only Gps Perturbated trade-offs of the sub-dataset

In [24]:
tradeOff[(tradeOff['dummyLocation'] == False) & (tradeOff['gpsPerturbated'] == True)].reset_index(drop=True)
# print(tradeOff[(tradeOff['dummyLocation'] == False) & (tradeOff['gpsPerturbated'] == True)].reset_index(drop=True).to_latex())

Index(['dummyLocation', 'gpsPerturbated', 'dumRadMin', 'dumRadStep', 'pertDec',
       'alpha', 'tradeOff'],
      dtype='object')
\begin{tabular}{lrrr}
\toprule
{} &  pertDec &  alpha &  tradeOff \\
\midrule
0   &        5 &  0.000 &  1.000000 \\
1   &        5 &  0.002 &  0.998000 \\
2   &        5 &  0.004 &  0.996000 \\
3   &        5 &  0.006 &  0.994000 \\
4   &        5 &  0.008 &  0.992000 \\
5   &        5 &  0.010 &  0.990000 \\
6   &        5 &  0.012 &  0.988000 \\
7   &        5 &  0.014 &  0.986000 \\
8   &        5 &  0.016 &  0.984000 \\
9   &        5 &  0.018 &  0.982000 \\
10  &        5 &  0.020 &  0.980000 \\
11  &        5 &  0.022 &  0.978000 \\
12  &        5 &  0.024 &  0.976000 \\
13  &        5 &  0.026 &  0.974000 \\
14  &        5 &  0.028 &  0.972000 \\
15  &        5 &  0.030 &  0.970000 \\
16  &        5 &  0.032 &  0.968000 \\
17  &        5 &  0.034 &  0.966000 \\
18  &        5 &  0.036 &  0.964000 \\
19  &        5 &  0.038 &  0.962000 \\
20  &      

Both Dummy Location and Gps Perturbated trade-offs of the subdataset

In [30]:
tradeOff[(tradeOff['dummyLocation'] == True) & (tradeOff['gpsPerturbated'] == True)].reset_index(drop=True)
# print(tradeOff[(tradeOff['dummyLocation'] == True) & (tradeOff['gpsPerturbated'] == True)].reset_index(drop=True).iloc[:,2:].to_latex(index=False))

\begin{tabular}{rrrrr}
\toprule
 dumRadMin &  dumRadStep &  pertDec &  alpha &     tradeOff \\
\midrule
       500 &         250 &        5 &  0.000 & 1.384329e-26 \\
      3000 &        2000 &        1 &  0.002 & 2.000000e-03 \\
      3000 &        2000 &        1 &  0.004 & 4.000000e-03 \\
      3000 &        2000 &        1 &  0.006 & 6.000000e-03 \\
      3000 &        2000 &        1 &  0.008 & 8.000000e-03 \\
      3000 &        2000 &        1 &  0.010 & 1.000000e-02 \\
      3000 &        2000 &        1 &  0.012 & 1.200000e-02 \\
      3000 &        2000 &        1 &  0.014 & 1.400000e-02 \\
      3000 &        2000 &        1 &  0.016 & 1.600000e-02 \\
      3000 &        2000 &        1 &  0.018 & 1.800000e-02 \\
      3000 &        2000 &        1 &  0.020 & 2.000000e-02 \\
      3000 &        2000 &        1 &  0.022 & 2.200000e-02 \\
      3000 &        2000 &        1 &  0.024 & 2.400000e-02 \\
      3000 &        2000 &        1 &  0.026 & 2.600000e-02 \\
      3000 &  